In [1]:
import numpy as np;
import pandas as pd;
import plotly.express as px;


In [2]:
# Global parameters
NO_OF_DAYS = 200
modelParameter = {}
modelParameter["beta"] = 0.49  
modelParameter["tE"] = 5
modelParameter["tI"] = 5
modelParameter["tH"] = 9
modelParameter["mu"] = 0.001

print(modelParameter)

# System variables
ROOT_DIR = "/Users/sreeramh/Documents/Kerala_Covid/SpatioTemporal/"

{'beta': 0.49, 'tE': 5, 'tI': 5, 'tH': 9, 'mu': 0.001}


In [9]:
# Load data
initDataDF = pd.read_csv(ROOT_DIR + "data/ksg_init_data.csv")
print(initDataDF)

neighborDF = pd.read_csv(ROOT_DIR + "data/ksg_neighbor_data.csv")
print(neighborDF)


                    name      S  E   I  H  R
0             Kanhangadu  73342  0  23  0  0
1               Chengala  56781  0   0  0  0
2                Chemnad  54747  0   0  0  0
3              Kasargodu  54172  0   0  0  0
4                Ajanoor  49153  0   0  0  0
5             Mangalpady  48441  0   0  0  0
6                Kumbala  46691  0   0  0  0
7             Pallikkara  43255  0   0  0  0
8           Manjewswaram  41515  0   0  0  0
9                 Madhur  41463  0   0  0  0
10           Nileshwaram  39752  0   0  0  0
11          Thrikkaripur  38687  0   0  0  0
12                  Udma  37537  0   0  0  0
13             Paivalike  34274  0   0  0  0
14             Badiyadka  34207  0   0  0  0
15           Kodombellur  33211  0   0  0  0
16          Pullurperiya  29642  0   0  0  0
17            West Eleri  29316  0   0  0  0
18              Bedaduka  27868  0   0  0  0
19  Kinanoor Karinthalam  27585  0   0  0  0
20           Cheruvathur  27435  0   0  0  0
21        

In [12]:
## Differential equation
def deriv(y, Ni, modelParameter):
    Si, Ei, Ii, Hi, Ri = y
    ## TODO what to do with contactRatio;
    dSdt = -modelParameter["beta"] * Si * Ii / Ni
    dEdt = (modelParameter["beta"] * Si * Ii / Ni) - (Ei/modelParameter["tE"])
    dIdt = (Ei/modelParameter["tE"]) -  (Ii/modelParameter["tI"])
    dHdt = (Ii/modelParameter["tI"]) - (Hi/modelParameter["tH"])
    dRdt = (Hi/modelParameter["tH"])
    return dSdt, dEdt, dIdt, dHdt, dRdt

In [6]:
def findSpatialContactRatio(name, NDict, yDict, neighborDF):
    ## TODO how to compute this
    contactRatio = 0;
    neighbors = neighborDF[neighborDF['fromName']==name]
    for index, row in neighbors.iterrows():
        #print(name, row['toName'], row['distance'], NDict[name], NDict[row['toName']])
        contactRatio = contactRatio + (mu * (1/row['distance']) * ((NDict[name] * NDict[row['toName']])/(NDict[name] + NDict[row['toName']])))
    return contactRatio/NDict[name];
        

In [7]:
## next count 
def derivNext(y, rates) :
    S,E,I,H, R = y
    dSdt, dEdt, dIdt, dHdt, dRdt = rates
    S1 = S + dSdt
    E1 = E + dEdt
    I1 = I + dIdt
    H1 = H + dHdt
    R1 = R + dRdt
    return S1, E1, I1, H1, R1


In [13]:
yDict = {};
NDict = {};
resultsDF = pd.DataFrame(columns=['name', 'day', 'S', 'E', 'I', 'H', 'R'], );

for index, row in initDataDF.iterrows():
    NDict[row['name']] = row['S'] + row['E'] + row['I'] + row['H'] + row['R']
    yDict[row['name']] = row['S'], row['E'], row['I'], row['H'], row['R']
    

for day in range(1,NO_OF_DAYS):
    for index, row in initDataDF.iterrows():
        name = row['name'];
        y = yDict[name]; 
        
        #contactRatio = findSpatialContactRatio(name, NDict, yDict, neighborDF);
        #print(name, "contactRatio = ", contactRatio);
        
        rates = deriv(y, NDict[name], modelParameter)
        yNext = derivNext(y, rates) 
        
        yDict[name] = yNext
        #print(name, yNext)
        resultsDF = resultsDF.append({'name': name, 'day': day, 'S': yNext[0], 'E': yNext[1], 'I': yNext[2], 'H': yNext[3], 'R': yNext[4]},ignore_index=True)


In [14]:
resultsDF

,name,day,S,E,I,H,R
0,Kanhangadu,1,73330.733533,11.266467,18.4,4.6,0.0
1,Chengala,1,56781.000000,0.000000,0.0,0.0,0.0
2,Chemnad,1,54747.000000,0.000000,0.0,0.0,0.0
3,Kasargodu,1,54172.000000,0.000000,0.0,0.0,0.0
4,Ajanoor,1,49153.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...
8154,Karadka,199,21211.000000,0.000000,0.0,0.0,0.0
8155,Kallar,199,19414.000000,0.000000,0.0,0.0,0.0
8156,Kumbadaje,199,14772.000000,0.000000,0.0,0.0,0.0
8157,Valiyaparamba,199,12790.000000,0.000000,0.0,0.0,0.0


In [15]:
resultsDF.to_csv(ROOT_DIR + "output/results.csv")

In [17]:
fig = px.scatter(resultsDF, x="day", y="H", color="name")
fig.show()